# 取得台股資料
每日收盤行情(全部(不含權證、牛熊證)) <br>
https://www.twse.com.tw/zh/page/trading/exchange/MI_INDEX.html

In [1]:
import pandas as pd
import datetime
import time

def stock_crawler(date):
    stock = pd.read_html("https://www.twse.com.tw/exchangeReport/MI_INDEX?response=html&date=" + date + "&type=ALLBUT0999")
    stock = stock[-1]
    stock.columns = ['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', 
                 '最高價', '最低價', '收盤價', '漲跌(+/-)', '漲跌價差', 
                 '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比']
    return stock

def trans_date(date):
    return str(date).split()[0].replace("-", "")

def parse_n_day(date, n):
    dic = {}
    for i in range(n):
        time.sleep(1)
        date = date - datetime.timedelta(days=1)
        try:
            df = stock_crawler(trans_date(date))
            dic.update({trans_date(date): df})
        except:
            continue
    return dic

In [2]:
result_dict = parse_n_day(datetime.datetime.now(), 10)
# result_dict
result_dict['20211230']

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,6421410,5369,935379902,145.95,146.00,145.35,145.50,-,0.45,145.45,20,145.50,98,0.00
1,0051,元大中型100,47256,83,2877292,61.00,61.00,60.80,60.80,-,0.10,60.80,47,60.85,1,0.00
2,0052,富邦科技,479557,251,64233789,134.40,134.40,133.80,134.15,-,0.05,134.00,1,134.15,3,0.00
3,0053,元大電子,2178,8,152396,70.00,70.00,69.95,69.95,-,0.05,69.95,1,70.00,1,0.00
4,0054,元大台商50,2728,10,86296,31.62,31.68,31.62,31.68,-,0.02,31.68,1,31.69,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,9944,新麗,189527,90,4542692,24.00,24.05,23.80,24.05,+,0.05,24.00,10,24.05,9,72.88
1145,9945,潤泰新,3354197,2537,213658721,63.80,64.20,63.30,63.80,+,0.30,63.80,36,63.90,126,9.00
1146,9946,三發地產,97928,58,1478443,15.10,15.15,15.05,15.10,NaN,0.00,15.05,24,15.10,27,35.95
1147,9955,佳龍,65217,156,1125342,17.25,17.30,17.20,17.25,+,0.05,17.20,4,17.25,4,0.00


# 建立以日期為一張表格的資料庫

In [3]:
import sqlite3

dbname = 'TWstock.db'
db = sqlite3.connect(dbname)

for i in result_dict.keys():
    result_dict[i].to_sql(i, db, if_exists='replace')

In [4]:
pd.read_sql(con=db, sql='SELECT * FROM "20211230"')

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0,0050,元大台灣50,6421410,5369,935379902,145.95,146.00,145.35,145.50,-,0.45,145.45,20,145.50,98,0.00
1,1,0051,元大中型100,47256,83,2877292,61.00,61.00,60.80,60.80,-,0.10,60.80,47,60.85,1,0.00
2,2,0052,富邦科技,479557,251,64233789,134.40,134.40,133.80,134.15,-,0.05,134.00,1,134.15,3,0.00
3,3,0053,元大電子,2178,8,152396,70.00,70.00,69.95,69.95,-,0.05,69.95,1,70.00,1,0.00
4,4,0054,元大台商50,2728,10,86296,31.62,31.68,31.62,31.68,-,0.02,31.68,1,31.69,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,1144,9944,新麗,189527,90,4542692,24.00,24.05,23.80,24.05,+,0.05,24.00,10,24.05,9,72.88
1145,1145,9945,潤泰新,3354197,2537,213658721,63.80,64.20,63.30,63.80,+,0.30,63.80,36,63.90,126,9.00
1146,1146,9946,三發地產,97928,58,1478443,15.10,15.15,15.05,15.10,None,0.00,15.05,24,15.10,27,35.95
1147,1147,9955,佳龍,65217,156,1125342,17.25,17.30,17.20,17.25,+,0.05,17.20,4,17.25,4,0.00


# 建立以證券代號為一張表格的資料庫

In [5]:
stock_df = pd.DataFrame()

for i in result_dict.keys():
    result_dict[i]['date'] = pd.to_datetime(i)
    stock_df = stock_df.append(result_dict[i])
# stock_df

In [6]:
dbname2 = 'TWstock2.db'
db2 = sqlite3.connect(dbname2)

stock_dict = dict(tuple(stock_df.groupby('證券代號')))
# print(stock_dict)
# print(stock_dict['0050'])

for i in stock_dict.keys():
    stock_dict[i].to_sql(i, db2, if_exists='replace')

In [7]:
pd.read_sql(con=db2, sql='SELECT * FROM "2330"')

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,date
0,438,2330,台積電,57490736,53430,36817638522,638.00,646.00,636.00,644.00,-,6.0,643.00,24.0,644.00,183.0,29.14,2022-01-06 00:00:00
1,438,2330,台積電,72505550,64712,47582832784,669.00,669.00,646.00,650.00,-,6.0,649.00,123.0,650.00,1030.0,29.41,2022-01-05 00:00:00
2,438,2330,台積電,90945643,106409,59188199534,645.00,656.00,644.00,656.00,+,25.0,656.00,66.0,657.00,803.0,29.68,2022-01-04 00:00:00
3,438,2330,台積電,73703302,88508,46249716919,619.00,632.00,618.00,631.00,+,16.0,630.00,109.0,631.00,211.0,28.55,2022-01-03 00:00:00
4,438,2330,台積電,20522055,15565,12654016133,619.00,620.00,615.00,615.00,-,1.0,615.00,536.0,616.00,144.0,27.83,2021-12-30 00:00:00
5,438,2330,台積電,25604320,28804,15788043027,615.00,619.00,614.00,616.00,+,1.0,616.0,405.0,617.00,141.0,27.87,2021-12-29 00:00:00
6,438,2330,台積電,35156339,36241,21569506434,610.00,615.00,610.00,615.00,+,9.0,614.0,250.0,615.00,2061.0,27.83,2021-12-28 00:00:00
